## WIP!
Will update with sexy examples when Andrew's code is in and everything is fast!

In [54]:
# Re-import modified modules without restarting the server
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
import os
import sys
import warnings

notebook_path = os.path.abspath(get_ipython().starting_dir)
parent_path = os.path.dirname(notebook_path)

sys.path.append(parent_path)
warnings.simplefilter("ignore")

In [49]:
# !git clone https://github.com/leap-laboratories/PIZZA.git
# !pip install python-dotenv openai transformers

In [50]:
from attribution.api_attribution import OpenAIAttributor
from attribution.experiment_logger import ExperimentLogger
from attribution.token_perturbation import (
    FixedPerturbationStrategy,
)

attributor = OpenAIAttributor()
# attributor = OpenAIAttributor(openai_api_key="YOUR_API_KEY_GOES_HERE")

In [51]:
input_str = "Repeat the word 'the' 3 times."
print(attributor.get_chat_completion(input_str).message.content)

the the the


In [52]:
logger = ExperimentLogger()
attributor.compute_attributions(
    input_str,
    perturbation_strategy=FixedPerturbationStrategy(""),
    attribution_strategies=["prob_diff"],
    logger=logger,
    perturb_word_wise=False,
    ignore_output_token_location=True,
)
display(logger.df_experiments)
logger.print_sentence_attribution()
logger.print_attribution_matrix(exp_id=1, show_debug_cols=True)

,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration
0,1,Repeat the word 'the' 3 times.,the the the,fixed,False,60.435367


,exp_id,attribution_strategy,perturbation_strategy,perturb_word_wise,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9
0,1,prob_diff,fixed,False,Repeat 0.11,the -0.03,word -0.02,' 0.02,the 0.29,' -0.01,3 -0.04,times 0.08,. -0.03


Attribution matrix for experiment 1 
Attribution Strategy: prob_diff 
Perturbation strategy: fixed:
Input Tokens (Rows) vs. Output Tokens (Columns)


,the (0),the (1),the (2),perturbed_input,perturbed_output
Repeat (0),0.345613,-0.005029,0.000544,the word 'the' 3 times.,the the the
the (1),-0.074411,-0.005559,0.000014,Repeat word 'the' 3 times.,the the the
word (2),-0.046202,-0.005560,0.000013,Repeat the 'the' 3 times.,the the the
' (3),0.073115,-0.005570,0.000003,Repeat the word the' 3 times.,the the the
the (4),0.876375,-0.005571,0.000002,Repeat the word'' 3 times.,Repeat the word repeat 3 times. Repeat the word repeat 3 times. Repeat the word repeat 3 times.
' (5),-0.034374,-0.005541,0.000032,Repeat the word 'the 3 times.,the the the
3 (6),-0.118118,-0.005534,0.000038,Repeat the word 'the' times.,the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
times (7),0.250415,-0.005569,0.000004,Repeat the word 'the' 3.,the the the
. (8),-0.069915,-0.005573,-0.000000,Repeat the word 'the' 3 times,the the the


In [53]:
logger = ExperimentLogger()
attributor.compute_attributions(
    input_str,
    perturbation_strategy=FixedPerturbationStrategy(""),
    attribution_strategies=["cosine"],
    logger=logger,
    perturb_word_wise=False,
    ignore_output_token_location=True,
)
display(logger.df_experiments)
logger.print_sentence_attribution()
logger.print_attribution_matrix(exp_id=1, show_debug_cols=True)

,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration
0,1,Repeat the word 'the' 3 times.,the the the,fixed,False,72.727495


,exp_id,attribution_strategy,perturbation_strategy,perturb_word_wise,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9
0,1,cosine,fixed,False,Repeat -0.00,the -0.00,word -0.00,' -0.00,the 0.84,' -0.00,3 -0.00,times -0.00,. -0.00


Attribution matrix for experiment 1 
Attribution Strategy: cosine 
Perturbation strategy: fixed:
Input Tokens (Rows) vs. Output Tokens (Columns)


,the (0),the (1),the (2),perturbed_input,perturbed_output
Repeat (0),-0.000000,0.000000,0.000000,the word 'the' 3 times.,the the the
the (1),-0.000000,0.000000,0.000000,Repeat word 'the' 3 times.,the the the
word (2),-0.000000,0.000000,0.000000,Repeat the 'the' 3 times.,the the the
' (3),-0.000000,0.000000,0.000000,Repeat the word the' 3 times.,the the the
the (4),0.764635,0.873588,0.873588,Repeat the word'' 3 times.,Repeat repeat repeat
' (5),-0.000000,0.000000,0.000000,Repeat the word 'the 3 times.,the the the
3 (6),-0.000000,0.000000,0.000000,Repeat the word 'the' times.,the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
times (7),-0.000000,0.000000,0.000000,Repeat the word 'the' 3.,the the the
. (8),-0.000000,0.000000,0.000000,Repeat the word 'the' 3 times,the the the
